In [ ]:
import copy
import optuna
import numpy as np
import matplotlib.pyplot as plt
from bcmix import *

In [ ]:
N_TRIALS = 20
DATA_LEN = 81
P = 0.025

In [ ]:
# true value
mean_true = np.array([[0.0], [2.0]])
covm_true = np.array([[9.0, 0.0], [0.0, 1.0]])
alpha, beta = np.random.multivariate_normal(mean_true.flatten(), covm_true)

# prior
canonical_0 = np.array([[0.0], [0.0]])
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])
logcon_0 = (np.linalg.slogdet(precision_0)[1] - (canonical_0.T @ np.linalg.inv(precision_0) @ canonical_0).item()) / 2
states = {0: {"can": canonical_0, "pre": precision_0, "log": logcon_0, "pit": 0.0}}

print(alpha, beta, myopic_mix(states, P))

### Rollout

In [ ]:
def objective_bcmix(trial):
    a = trial.suggest_float('a', ACTION_RANGE_BCMIX[0], ACTION_RANGE_BCMIX[1])
    q = q_myopic_with_change(states_i, a, P)
    return q

def objective(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_without_change(canonical_i, precision_i, a)
    return q

In [ ]:
for i in range(1):
    # initialize
    simresult_bcmix_i = np.full((DATA_LEN, 6 * (1 + M1 + M2) + 5), 0.0)
    simresult_i = np.full((DATA_LEN, 10), np.nan)
    alpha_i, beta_i = alpha, beta
    canonical_i, precision_i, states_i = canonical_0, precision_0, copy.deepcopy(states)
    # change point locations
    """
    cp_j = [np.random.randint(10, 71)] if (i // 2) else [np.random.randint(5, 31), np.random.randint(50, 76)]
    """
    cp_j = [3, 50]
    for j in range(DATA_LEN):
        # current state
        # bcmix model
        for m, s in states_i.items():
            simresult_bcmix_i[j, 0] = alpha_i
            simresult_bcmix_i[j, 1] = beta_i
            covm_bcmix_i = np.linalg.inv(s["pre"])
            mean_bcmix_i = covm_bcmix_i @ s["can"]
            simresult_bcmix_i[j, m * 6 + 2] = mean_bcmix_i[0][0]
            simresult_bcmix_i[j, m * 6 + 3] = mean_bcmix_i[1][0]
            simresult_bcmix_i[j, m * 6 + 4] = covm_bcmix_i[0][0]
            simresult_bcmix_i[j, m * 6 + 5] = covm_bcmix_i[0][1]
            simresult_bcmix_i[j, m * 6 + 6] = covm_bcmix_i[1][1]
            simresult_bcmix_i[j, m * 6 + 7] = s["pit"]
        # old model
        simresult_i[j, 0] = alpha_i
        simresult_i[j, 1] = beta_i
        covm_i = np.linalg.inv(precision_i)
        mean_i = covm_i @ canonical_i
        simresult_i[j, 2] = mean_i[0][0]
        simresult_i[j, 3] = mean_i[1][0]
        simresult_i[j, 4] = covm_i[0][0]
        simresult_i[j, 5] = covm_i[0][1]
        simresult_i[j, 6] = covm_i[1][1]
        # select action
        # bcmix model
        x_m_bcmix = myopic_mix(states_i, P)[0]
        ACTION_RANGE_BCMIX = (x_m_bcmix - 5, x_m_bcmix + 5)
        study_bcmix = optuna.create_study(direction="maximize")
        study_bcmix.optimize(objective_bcmix, n_trials=N_TRIALS)
        a_bcmix = study_bcmix.best_trial.params['a']
        simresult_bcmix_i[j, -3] = a_bcmix
        simresult_bcmix_i[j, -2] = study_bcmix.best_trial.value
        # old model
        x_m = myopic(canonical_i, precision_i)[0]
        ACTION_RANGE = (x_m - 5, x_m + 5)
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=N_TRIALS)
        a = study.best_trial.params['a']
        simresult_i[j, 7] = a
        simresult_i[j, 8] = study.best_trial.value
        # update state
        if j in cp_j:
            alpha_i_ = alpha_i
            alpha_i, beta_i = np.random.multivariate_normal(mean_true.flatten(), covm_true)
            alpha_i = alpha_i if alpha_i * alpha_i_ < 0 else -1 * alpha_i
        # bcmix model
        y_bcmix = env_response(a_bcmix, alpha_i, beta_i)[0]
        simresult_bcmix_i[j, -1] = reward(a_bcmix, y_bcmix)
        states_i = update_with_change(states_i, a_bcmix, y_bcmix, P)
        # old model
        y = env_response(a, alpha_i, beta_i)[0]
        simresult_i[j, 9] = reward(a, y)
        canonical_i, precision_i = update_without_change(canonical_i, precision_i, a, y)
    np.save("simulations\sim_cpbcmix_" + str(i) + ".npy", simresult_bcmix_i)
    np.save("simulations\sim_cpold_" + str(i) + ".npy", simresult_i)


In [ ]:
for i in range(1):
    # initialize
    simresult_bcmix_i = np.load("simulations\sim_cpbcmix_" + str(i) + ".npy")
    simresult_myopic_i = np.full((DATA_LEN, 6 * (1 + M1 + M2) + 5), 0.0)
    states_i = copy.deepcopy(states)
    for j in range(DATA_LEN):
        # current state
        alpha_i, beta_i = simresult_bcmix_i[j, 0], simresult_bcmix_i[j, 1]
        for m, s in states_i.items():
            simresult_myopic_i[j, 0] = alpha_i
            simresult_myopic_i[j, 1] = beta_i
            covm_bcmix_i = np.linalg.inv(s["pre"])
            mean_bcmix_i = covm_bcmix_i @ s["can"]
            simresult_myopic_i[j, m * 6 + 2] = mean_bcmix_i[0][0]
            simresult_myopic_i[j, m * 6 + 3] = mean_bcmix_i[1][0]
            simresult_myopic_i[j, m * 6 + 4] = covm_bcmix_i[0][0]
            simresult_myopic_i[j, m * 6 + 5] = covm_bcmix_i[0][1]
            simresult_myopic_i[j, m * 6 + 6] = covm_bcmix_i[1][1]
            simresult_myopic_i[j, m * 6 + 7] = s["pit"]
        # select action
        a_myopic = myopic_mix(states_i, P)[0]
        simresult_myopic_i[j, -3] = a_myopic
        simresult_myopic_i[j, -2] = np.nan
        # update state
        y_myopic = env_response(a_myopic, alpha_i, beta_i)[0]
        simresult_myopic_i[j, -1] = reward(a_myopic, y_myopic)
        states_i = update_with_change(states_i, a_myopic, y_myopic, P)
    np.save("simulations\sim_cpmyopic_" + str(i) + ".npy", simresult_myopic_i)


### Plot

In [ ]:
reg_bcmix = np.full(DATA_LEN, 0.0)
mse_bcmix = np.full(DATA_LEN, 0.0)
reg_myopic = np.full(DATA_LEN, 0.0)
mse_myopic = np.full(DATA_LEN, 0.0)
reg = np.full(DATA_LEN, 0.0)
mse = np.full(DATA_LEN, 0.0)

for i in range(1):
    simresult_bcmix_i = np.load("simulations\sim_cpbcmix_" + str(i) + ".npy")
    simresult_myopic_i = np.load("simulations\sim_cpmyopic_" + str(i) + ".npy")
    simresult_i = np.load("simulations\sim_cpold_" + str(i) + ".npy")
    for j in range(DATA_LEN):
        alpha, beta = simresult_bcmix_i[j, 0], simresult_bcmix_i[j, 1]
        reg_bcmix[j] += (GAMMA ** j) * (alpha + simresult_bcmix_i[j, -3] * beta) ** 2
        reg_myopic[j] += (GAMMA ** j) * (alpha + simresult_myopic_i[j, -3] * beta) ** 2
        reg[j] += (GAMMA ** j) * (alpha + simresult_i[j, 7] * beta) ** 2
        for m in range(1 + M1 + M2):
            m_a = simresult_bcmix_i[j, m * 6 + 2]
            m_b = simresult_bcmix_i[j, m * 6 + 3]
            v_a = simresult_bcmix_i[j, m * 6 + 4]
            v_b = simresult_bcmix_i[j, m * 6 + 6]
            p_c = simresult_bcmix_i[j, m * 6 + 7]
            mse_bcmix[j] += (v_a + v_b + (m_a - alpha) ** 2 + (m_b - beta) ** 2) * (1 if j == 0 and m == 0 else p_c)
        for m in range(1 + M1 + M2):
            m_a = simresult_myopic_i[j, m * 6 + 2]
            m_b = simresult_myopic_i[j, m * 6 + 3]
            v_a = simresult_myopic_i[j, m * 6 + 4]
            v_b = simresult_myopic_i[j, m * 6 + 6]
            p_c = simresult_myopic_i[j, m * 6 + 7]
            mse_myopic[j] += (v_a + v_b + (m_a - alpha) ** 2 + (m_b - beta) ** 2) * (1 if j == 0 and m == 0 else p_c)
        m_a = simresult_i[j, 2]
        m_b = simresult_i[j, 3]
        v_a = simresult_i[j, 4]
        v_b = simresult_i[j, 6]
        mse[j] += v_a + v_b + (m_a - alpha) ** 2 + (m_b - beta) ** 2

reg_bcmix /= 1
mse_bcmix /= 1
reg_myopic /= 1
mse_myopic /= 1
reg /= 1
mse /= 1

In [ ]:
# plot regret
plt.figure(figsize=(7.2, 4.8))
plt.plot([sum(reg_bcmix[j:]) / (GAMMA ** j) for j in range(1, DATA_LEN)], 'k', label="BCMIX rollout")
plt.plot([sum(reg[j:]) / (GAMMA ** j) for j in range(1, DATA_LEN)], "k--", label="rollout")
plt.plot([sum(reg_myopic[j:]) / (GAMMA ** j) for j in range(1, DATA_LEN)], "k:", label="BCMIX myopic")
plt.legend(loc="upper right")
plt.title("regret")
plt.xlabel("decision time")
plt.ylabel("regret")
#plt.savefig("fig1", dpi=300)

In [ ]:
# plot mse
plt.figure(figsize=(7.2, 4.8))
plt.plot(mse_bcmix, 'k', label="BCMIX rollout")
plt.plot(mse, "k--", label="rollout")
plt.plot(mse_myopic, "k:", label="BCMIX myopic")
plt.legend(loc="upper right")
plt.title("MSE of parameter estimation")
plt.xlabel("decision time")
plt.ylabel("mse")
#plt.savefig("fig2", dpi=300)